<a href="https://colab.research.google.com/github/lovrodukic/music-recommendation/blob/main/music_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>